In [ ]:
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim

# Define the neural network architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.dropout = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = nn.functional.relu(nn.functional.max_pool2d(self.conv1(x), 2))
        x = nn.functional.relu(nn.functional.max_pool2d(self.dropout(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.dropout(x, training=self.training)
        x = self.fc2(x)
        return nn.functional.log_softmax(x, dim=1)

# Load the MNIST dataset and define the data loaders
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1000, shuffle=False)

# Initialize the neural network and the optimizer
net = Net()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.5)

# Train the neural network
for epoch in range(10):
    net.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = net(data)
        loss = nn.functional.nll_loss(output, target)
        loss.backward()
        optimizer.step()

    # Evaluate the neural network on the test data
    net.eval()
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = net(data)
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    accuracy = correct / len(test_loader.dataset)
    print('Epoch {} Test Accuracy: {:.2f}%'.format(epoch, 100. * accuracy))




Epoch 0 Test Accuracy: 94.23%
Epoch 1 Test Accuracy: 95.90%
Epoch 2 Test Accuracy: 96.59%
Epoch 3 Test Accuracy: 97.06%
Epoch 4 Test Accuracy: 97.36%
Epoch 5 Test Accuracy: 97.75%
Epoch 6 Test Accuracy: 97.87%
Epoch 7 Test Accuracy: 97.88%
Epoch 8 Test Accuracy: 98.04%
Epoch 9 Test Accuracy: 98.10%


In [ ]:
# Save the trained model
torch.save(net.state_dict(), 'mnist_model.pth')

# Download the saved model
from google.colab import files
files.download('mnist_model.pth')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>